In [ ]:
!pip install awswrangler

In [ ]:
import awswrangler as wr
import pandas as pd
from datetime import date

In [ ]:
date_for_path = date.today().strftime('%Y%m%d')
date_for_tbl = date.today()

input_path = 's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_etl_inputs/tam_pccm_interactions/ACM Interaction Summary Extract Jan 22 to May 8 24.csv'
output_path = 's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_etl_outputs/tam_pccm_interactions/'
df = pd.read_csv(input_path, dtype='str', header=0)

df = df.rename(columns={
    'Zip Code (Zip5)': 'zip_code',
    'Product': 'product',
    'Interaction Date': 'interaction_date',
    'Interaction Type': 'interaction_type',
    'Topic Discussed': 'topic_discussed',
    '# of Patients': 'number_of_patients',
    '# of Interactions': 'number_of_interactions'
})

df['number_of_patients'] = df['number_of_patients'].astype(float)
df['number_of_interactions'] = df['number_of_interactions'].astype(float)
df['etl_load_date'] = date_for_tbl


try: 
    wr.s3.to_parquet(
        df=df,
        path=f'{output_path}{date_for_path}/',
        dataset=True,
        database='sandbox_measurement_analyst',
        table='mars_tam_pccm_interactions_staging',
        mode = 'overwrite',
        index=False
    )
except Exception as e:
    raise e